# Digital diktanalyse: Mønstre i mengden

_18.februar 2026_

Velkommen til kodeverksted på Nasjonalbiblioteket!

## Innhold

I løpet av dette verkstedet skal vi annotere ulike gjentakelsesmønstre i digitale tekster med regelbaserte algoritmer.
Koden vi kommer til å bruke er fra et kodebibliotek for [`python`](https://www.python.org) som heter [`poetry_analysis`](https://pypi.org/project/poetry-analysis/).

Vi skal gå gjennom tre forskjellige lyriske virkemidler som baserer seg på gjentakelser: 
1. [Enderim](#enderim)
2. [Bokstavrim](#bokstavrim) 
3. [Anafor](#anafor)

Med egne funksjoner for hvert av trekkene kan du blant annet undersøke om det ofte er de samme ordene som rimer på hverandre i enderimsmønstre, om bokstavrim dannes med vokaler eller konsonanter,  og hvilke ord som oftest brukes anaforisk.

### Visualisering

I tillegg til kodebiblioteket, som kan brukes av alle med en pythonversjon høyere enn 3.11, har vi  utviklet en egen app der man kan eksperimentere med enkelttekster og visualisere annotasjonene av gjentakelsesmønstrene: 

https://dh.nb.no/run/diktanalyse/app/

<!--
Som eksempeldikt til demoen har jeg valgt ut et dikt som passer måneden vi er inne i:  
-->
## Eksempeldikt
> "Februarvise" fra *Høstblomster eller rimstubber* (1896), av Andreas Aabel.

In [ ]:
from pathlib import Path

filepath = Path("eksempeldikt/Februarvise.txt")
text = filepath.read_text()

print(text)

## Enderim

Enderim definerer vi som stavelsesrim på slutten av en verselinje, altså at siste stavelse på én verselinje rimer på siste stavelse i en annen verselinje. 

Les mer om rim i *Store norske leksikon:* https://snl.no/rim

<!--
I praksis er koden implementert slik:
- Vi går gjennom alle verselinjene i en strofe eller et dikt 
- For hver linje starter vi på bakerste bokstav, og sammenligner bokstav for bokstav, med siste ordet på alle tidligere linjer i strofen
- Dersom det er overlapp mellom to ord, og den delen som overlapper inneholder en vokal, så sier vi at ordene rimer.
- Da annoterer vi disse linjene med en merkelapp fra alfabetet.
- Sånn får vi mønstre som ABBA, AABB og ABAB i strofer på fire linjer der to og to av linjene rimer. 
-->

In [ ]:
from poetry_analysis.rhyme_detection import tag_text

# Annoter diktet i tekstfilen med funksjonen "tag_text"
end_rhymes = list(tag_text(text))

end_rhymes

Resultatet er en liste med annotasjoner for hver strofe, som igjen inneholder en liste med annotasjoner per linje. 

Ved å flate ut den hierarkiske datastrukturen kan vi  laste inn annotasjonene i en dataramme og se nærmere på dem: 

In [ ]:
import pandas as pd

def rhymes_to_df(rhyme_annotations:list) -> pd.DataFrame:
    """Flat ut listen med enderimsannotasjoner og last inn i en dataramme"""
    flattened_data = []
    for stanza in end_rhymes:
        stanza_id = stanza['stanza_id']
        rhyme_scheme = stanza['rhyme_scheme']
        for verse in stanza['verses']:
            flat_dict = verse.copy()
            # Disse to linjene bare rydder opp litt og fjerner noen felter vi ikke trenger
            del flat_dict["transcription"]
            del flat_dict["syllables"]
            # Legg til informasjon om strofen på hver av verselinjene
            flat_dict.update({
                'stanza_id': stanza_id,
                'rhyme_scheme': rhyme_scheme,
                'poem_id':1  # Legg til en provisorisk ID for diktet
            })
            flattened_data.append(flat_dict)

    # Lag en pandas dataramme
    return pd.DataFrame(flattened_data)

rhyme_df = rhymes_to_df(end_rhymes)

rhyme_df

### Rimordpar

Finnes det noen rimord som forekommer oftere enn andre i eksempeldiktet vårt? 

In [ ]:
def find_rhyming_word(row: pd.Series, df: pd.DataFrame) -> str | None:
    """For hver linje, hent siste ordet fra linjen som den rimer på."""
    rhyming_word = df["last_token"].to_numpy()[
        (df["poem_id"] == row["poem_id"])
        & (df["stanza_id"] == row["stanza_id"])
        & (df["verse_id"] == row["rhymes_with"])
    ]
    return rhyming_word.item() if rhyming_word.any() else None

def find_frequent_rhyme_pairs(df: pd.DataFrame) -> pd.Series:
    """Legg til kolonnen "rhyming_word" i datarammen `df` og tell frekvensen til rimordpar."""
    df.loc[:, "verse_id"] = df["verse_id"].astype(float)
    df["rhyming_word"] = df.apply(
        lambda row: find_rhyming_word(row, df), axis=1
    )
    # Grupper sammen siste ord i hver linje og ordet det rimer på, og sorter på frekvens til rimordparene
    rhyming_pairs = df[
        [
            "verse_id",
            "rhymes_with",
            "rhyming_word",
            "last_token",
        ]
    ][df["rhyming_word"].notna()].groupby("rhyming_word")["last_token"].value_counts().sort_values(ascending=False)
    return rhyming_pairs


rhyme_pairs = find_frequent_rhyme_pairs(rhyme_df)
rhyme_pairs.to_csv("utdata/rimord_frekvens_eksempeldikt.csv")

rhyme_pairs

## Bokstavrim

<!-- Bokstavrim: konsonantisk og vokalisk (men ikke assonans!)  -->
Bokstavrim, eller alliterasjon, definerer vi som gjentakelsen av en bokstav eller en språklyd i begynnelsen av påfølgende ord. 
Med `poetry_analysis` kan vi skille mellom bokstavrim som begynner på samme konsonant eller vokal, eller hente ut alle. 
Vi har ikke implementert kode for å fange opp såkalt "vokalisk alliterasjon" eller det som kalles assonans, eller halvrim.

Les mer om alliterasjon og assonans: 
- https://snl.no/allitterasjon
- https://snl.no/assonans

### Finn alle bokstavrim

In [ ]:
from poetry_analysis.alliteration import find_line_alliterations

# Vi gjenbruker eksempeldiktet vi lastet inn tidligere i variabelen "text"
alliterations = find_line_alliterations(text, letter_type="both")

alliterations

In [ ]:
from collections import Counter

# Tell antall ord
unique_words_in_alliterations = len(set([word for words in alliterations for word in words]))
print("Antall unike ord som inngår i bokstavrim i dette diktet: ", unique_words_in_alliterations)

# Finn ordrekka med flest bokstavrim på samme bokstav
top_count = max([len(words) for words in alliterations])
print("Største antall ord som inngår i samme bokstavrim: ", top_count)

letters= Counter([word[0] for line in alliterations for word in line])
print("Bokstaver som inngår i bokstavrimene: " )
letters

### Finn bare bokstavrim som begynner på konsonanter

In [ ]:
from poetry_analysis.alliteration import find_line_alliterations

# Vi gjenbruker eksempeldiktet vi lastet inn tidligere i variabelen "text"
alliterations = find_line_alliterations(text, letter_type="consonant")

alliterations

### Finn bare bokstavrim som begynner på vokaler

In [ ]:
alliterations = find_line_alliterations(text, letter_type="vowel")

alliterations

## Anafor

Anafor definerer vi som gjentakende ord eller fraser i begynnelsen av påfølgende verselinjer. 

Les mer om anafor: https://snl.no/anafor

Funksjonen `extract_anaphora` henter ut henholdsvis uni-, bi-, tri- og firegrammer fra påfølgende linjer

In [ ]:
from poetry_analysis.anaphora import extract_anaphora

anaphora_ngrams = extract_anaphora(text)
anaphora_ngrams

Funksjonen `extract_line_anaphora` henter ut fraser som gjentar seg på samme linje

In [ ]:
from poetry_analysis.anaphora import extract_line_anaphora

line_anaphora = extract_line_anaphora(text)

line_anaphora

## Hvordan undersøke mange tekster samtidig?  

Vi kan jo lett se gjentakelsesmønstrene selv når vi ser på én tekst av gangen, men hva om du har tusenvis? 

Ta for eksempel korpuset [NORN Dikt](https://github.com/norn-uio/norn-poems) som har over 3000 korrekturleste dikt og som jeg har lastet inn i en jsonl-fil:  `norn_poems.jsonl`. 
Filen `metadata.jsonl` inneholder bibliografisk informasjon om bøkene som diktene er publisert i. 

In [ ]:
import pandas as pd

# Last inn i en pandas dataramme
textdata = pd.read_json("norn_poems.jsonl", lines=True)
meta = pd.read_json("metadata.jsonl", lines=True)

poem_df = meta.merge(textdata, on="poem_id")

### Enderim på mange tekster

Så kan vi bruke funksjonen `tag_text` på datarammen for å annotere enderimene: 

In [ ]:
from poetry_analysis.rhyme_detection import tag_text

poem_df["end_rhymes"] = poem_df.text.apply(lambda t: list(tag_text(t)))

Resultatet er et nøstet hierarki av strofer og linjer og annotasjoner som hører til, så vi flater det ut i en ny dataramme: 

In [ ]:
def flatten_df_rhyme_annotations(df):
    all_records = []

    for _, row in df.iterrows():
        # Normaliser annotasjonsstrukturen for hver rad
        normalized = pd.json_normalize(
            row["end_rhymes"],
            record_path="verses",
            meta=["stanza_id", "rhyme_scheme"],
        )
        # Legg til dikt-ID så vi beholder koblingen til metadata
        normalized["poem_id"] = row["poem_id"]
        all_records.append(normalized)

    # Slå sammen alle radene
    expanded = pd.concat(all_records, ignore_index=True)
    return expanded

rhyme_df = flatten_df_rhyme_annotations(poem_df)

Deretter kan vi sjekke hvilke ordpar som rimer oftest i hele tekstsamlingen med over 3000 dikt

In [ ]:
# OBS! Det tar over 15 minutter å kjøre denne koden
rhyme_pairs = find_frequent_rhyme_pairs(rhyme_df)

rhyme_pairs.to_csv("utdata/rimord_frekvens_norn_poems.csv")

### Hvilke anaforer forekommer oftest?


In [ ]:

from poetry_analysis.anaphora import construct_anaphora_df

anaphora_df = construct_anaphora_df(rhyme_df, anaphora_length=3)

In [ ]:
anaphora_df.phrase.value_counts()

## Hvordan undersøke egne tekster?  

Dersom du har lagret tekstene dine i egne `.txt`-filer, én fil per tekst, er det lett å laste dem inn i en dataramme og gå frem på lik måte som vi har gjort i cellene ovenfor. 


In [ ]:
# 1: Definer filstien til mappen med tekstfiler som en variabel
dirpath = "eksempeldikt"

results = []
# Gå gjennom hver fil i mappen, én av gangen
for file in Path(dirpath).iterdir(): 
    # Last inn teksten fra filen
    text = file.read_text()
    # Annoter enderim i teksten
    rhymes = tag_text(text)
    # Lagre annotasjonene i resultat-listen
    results.append(rhymes)

results